<a href="https://colab.research.google.com/github/mnaseri94/Sleep-Wake-DL/blob/main/machine_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Google Drive**


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np


from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import  TimeSeriesSplit
from sklearn.metrics import mean_squared_error

In [ ]:
data = pd.read_excel('/content/drive/MyDrive/Deep-learning-howsam/RNN/scores.xlsx')
data.iloc[: , 1:].head()

,sleep duration,sleep efficiency,num_interruptions,sleep score
0,0,0,1,1
1,0,0,2,2
2,0,0,2,2
3,0,0,1,1
4,0,0,2,2


In [ ]:
data_sleep_scores = data.iloc[: , -1]

In [ ]:
normalize_sleep_scores = []
for score in data_sleep_scores:
    norm_score = (score - min(data_sleep_scores)) / (max(data_sleep_scores) - min(data_sleep_scores))
    normalize_sleep_scores.append(norm_score)

normalize_sleep_scores = np.array(normalize_sleep_scores)

In [ ]:
time_step = 6
offset = 1

X = []
y = []
for i in range(0 , len(normalize_sleep_scores)-time_step , offset):
    X.append(normalize_sleep_scores[i:i+time_step])
    y.append(normalize_sleep_scores[i+time_step])
X = np.array(X)
y = np.array(y)

print(X.shape)
print(y.shape)

(53, 6)
(53,)


In [ ]:
scores = {}
n_splits = 50


tscv = TimeSeriesSplit(n_splits = n_splits)
for train_index , valid_index in tscv.split(X):
    # train Subset
    X_train = X[train_index]
    y_train = y[train_index]

    # Validation
    X_valid = X[valid_index]
    y_valid = y[valid_index]

    # Random Forest
    rf_model = RandomForestRegressor(n_estimators=100 , max_depth=3 ,
                                  ccp_alpha=0.3)
    rf_model.fit(X_train, y_train)
    y_pred = rf_model.predict(X_valid)
    scores['Random_Forest'] = round(mean_squared_error(y_pred , y_valid) , 6)

    # svm.SVR
    svr_model = SVR()
    svr_model.fit(X_train, y_train)
    y_pred = svr_model.predict(X_valid)
    scores['SVM'] = round(mean_squared_error(y_pred , y_valid) , 6)

    # Linear Regression
    reg_model = SGDRegressor()
    reg_model.fit(X_train, y_train)
    y_pred = reg_model.predict(X_valid)
    scores['Linear Regression'] = round(mean_squared_error(y_pred , y_valid) , 6)

scores

{'Random_Forest': 0.009725, 'SVM': 0.001916, 'Linear Regression': 0.006217}